<a href="https://colab.research.google.com/github/ggvaldez/LGPD-Script/blob/main/LGPD_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Defina aqui a URL da planilha a ser utilizada para a LGPD

In [ ]:
url_google_sheets = 'https://docs.google.com/spreadsheets/d/DOCUMENTO_AQUI' # Insira a url dentro das aspas

Esse trecho autentica na sua conta Google e acessa a Planilha


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url(url_google_sheets)
sheet = wb.worksheet('Data Gather')

Esse trecho executa o script na planilha


Esse trecho exibe os erros encontrados na Planilha

Esse trecho gera o esboço do Documento
